In [47]:
@file:DependsOn("com.google.adk:google-adk:0.3.0")
@file:DependsOn("com.google.adk:google-adk-dev:0.2.0")
@file:DependsOn("com.google.genai:google-genai:1.23.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-rx3:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.8.1")

In [33]:
import com.google.adk.agents.LlmAgent
import com.google.adk.agents.SequentialAgent
import com.google.adk.agents.ParallelAgent
import com.google.adk.agents.LoopAgent

import com.google.adk.models.Gemini
import com.google.genai.Client

import com.google.adk.runner.InMemoryRunner
import com.google.adk.sessions.Session
import com.google.adk.sessions.InMemorySessionService
import com.google.adk.memory.InMemoryMemoryService
import com.google.adk.events.Event
import com.google.adk.tools.FunctionTool
import com.google.adk.tools.GoogleSearchTool

import com.google.genai.types.Content
import com.google.genai.types.Part

import java.util.UUID
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.reactive.asFlow

import io.reactivex.rxjava3.core.Flowable;


In [34]:
val apiKey = System.getenv("GOOGLE_API_KEY")
if (apiKey == null) {
    throw IllegalStateException("Please set the GOOGLE_API_KEY environment variable.")
}

In [35]:
val geminiClient = Client.builder().apiKey(apiKey).build()

In [36]:
val geminiModel = Gemini(
    "gemini-2.5-flash-lite", // Your model name
    geminiClient
)

In [37]:
val initialWriterAgent = LlmAgent.builder()
    .name("InitialWriterAgent")
    .model(geminiModel)
    .instruction("""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
Output only the story text, with no introduction or explanation.""")
    .outputKey("current_story")
    .build()

println("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [38]:
val criticAgent = LlmAgent.builder()
    .name("CriticAgent")
    .model(geminiModel)
    .instruction("""You are a constructive story critic. Review the story provided below.
Story: {current_story}

Evaluate the story's plot, characters, and pacing.
- If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
- Otherwise, provide 2-3 specific, actionable suggestions for improvement.""")
    .outputKey("critique")
    .build()

println("✅ critic_agent created.")

✅ critic_agent created.


In [39]:
object StoryTools {
    /**
     * Call this function ONLY when the critique is 'APPROVED',
     * indicating the story is finished and no more changes are needed.
     */
    fun exitLoop(): Map<String, String> {
        return mapOf(
            "status" to "approved",
            "message" to "Story approved. Exiting refinement loop."
        )
    }
}

println("✅ StoryTools object with exitLoop() function created.")

✅ StoryTools object with exitLoop() function created.


In [40]:
val exitLoopTool = FunctionTool.create(
    StoryTools,
    "exitLoop"
)

val refinerAgent = LlmAgent.builder()
    .name("RefinerAgent")
    .model(geminiModel) // Re-using the model from the first cell
    .instruction("""You are a story refiner. You have a story draft and critique.

    Story Draft: {current_story}
    Critique: {critique}

    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""")
    .outputKey("current_story")
    .tools(listOf(exitLoopTool))
    .build()

println("✅ refiner_agent created.")

✅ refiner_agent created.


In [41]:
// The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
val storyRefinementLoop = LoopAgent.builder()
    .name("StoryRefinementLoop")
    .subAgents(listOf(criticAgent, refinerAgent))
    .maxIterations(2) // Prevents infinite loops
    .build()

// The root agent is a SequentialAgent that defines the overall workflow:
// Initial Write -> Refinement Loop.
val rootAgent = SequentialAgent.builder()
    .name("StoryPipeline")
    .subAgents(listOf(initialWriterAgent, storyRefinementLoop))
    .build()

println("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [42]:
var runner: InMemoryRunner = InMemoryRunner(rootAgent, "Critiq Flow")


In [43]:
val prompt = "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"

var userMsg: Content? = Content.fromParts(Part.fromText(prompt))


In [44]:
var session: Session? = runner
    .sessionService()
    .createSession("Critiq Flow", "user")
    .blockingGet()


In [45]:
var events: Flowable<Event> = runner.runAsync("user", session.id(), userMsg)
println("Agent response ->")
events.blockingForEach{ event -> println(event.stringifyContent()) }

Agent response ->
Elias, lighthouse keeper for thirty years, knew the coast like the back of his calloused hand. The granite tower was his world, the rhythmic sweep of the lamp his heartbeat. One blustery evening, while cleaning the Fresnel lens, his rag snagged on something tucked deep within a crevice. He pulled out a vellum scroll, brittle with age, yet strangely supple. As he unrolled it, a faint, ethereal glow emanated from the ink. It wasn't ink, he realized, but a map. Lines of shimmering blue traced unknown coastlines, dotted with constellations that mirrored no sky he'd ever seen. In the center, a single, pulsating golden star marked a spot far beyond any charted ocean. A shiver, not entirely from the wind, traced its way down his spine.
This is a promising start to a story! The imagery is strong and immediately draws the reader into Elias's world. Here are a few suggestions for improvement:

1.  **Expand on Elias's character and his relationship with the lighthouse:** While w